In [11]:
import numpy as np
from scipy import interpolate
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import QuantLib as ql

#Let's create some random  data
array = np.random.randint(0,10,(10,10)).astype(float)
#values grater then 7 goes to np.nan
array[array>7] = np.nan

In [12]:
array

array([[ 1.,  7.,  4., nan,  6.,  2.,  0.,  6.,  1.,  5.],
       [ 2.,  6., nan, nan,  1.,  2., nan,  6.,  3.,  5.],
       [nan,  4.,  2.,  1., nan,  7.,  5.,  2.,  7.,  5.],
       [ 6.,  1.,  4.,  2., nan,  1.,  5.,  6.,  1.,  5.],
       [ 7.,  4.,  5.,  0.,  7.,  2.,  5.,  7., nan,  4.],
       [nan,  1.,  6.,  4.,  6., nan, nan,  3.,  0.,  2.],
       [ 0.,  0.,  5.,  5.,  3., nan,  7.,  2.,  5.,  5.],
       [ 6.,  2.,  6., nan,  4.,  2.,  0.,  6.,  0.,  1.],
       [ 0.,  4.,  1.,  0.,  6.,  3.,  7.,  4.,  1., nan],
       [ 7.,  6., nan,  4.,  6., nan,  5., nan,  0.,  5.]])

In [15]:
x = np.arange(0, array.shape[1])
y = np.arange(0, array.shape[0])
#mask invalid values
array = np.ma.masked_invalid(array)
xx, yy = np.meshgrid(x, y)
#get only the valid values
x1 = xx[~array.mask]
y1 = yy[~array.mask]
newarr = array[~array.mask]

GD1 = interpolate.griddata(
    (x1, y1), 
    newarr.ravel(),
    (xx, yy),
    method='cubic'
)
GD1

array([[1.        , 7.        , 4.        , 4.52202747, 6.        ,
        2.        , 0.        , 6.        , 1.        , 5.        ],
       [2.        , 6.        , 2.58745653, 0.71856306, 1.        ,
        2.        , 4.38437573, 6.        , 3.        , 5.        ],
       [3.68429175, 4.        , 2.        , 1.        , 3.78445571,
        7.        , 5.        , 2.        , 7.        , 5.        ],
       [6.        , 1.        , 4.        , 2.        , 1.34278083,
        1.        , 5.        , 6.        , 1.        , 5.        ],
       [7.        , 4.        , 5.        , 0.        , 7.        ,
        2.        , 5.        , 7.        , 3.47688958, 4.        ],
       [1.93900185, 1.        , 6.        , 4.        , 6.        ,
        6.6238854 , 6.99507679, 3.        , 0.        , 2.        ],
       [0.        , 0.        , 5.        , 5.        , 3.        ,
        5.4117267 , 7.        , 2.        , 5.        , 5.        ],
       [6.        , 2.        , 6.       

In [ ]:
from historical_alphaVantage_collection import ivol_df
ivol_df

In [ ]:
ivol_df = ivol_df.dropna(how='all',axis=0).dropna(how='all',axis=1)
print(f"\nT: \n{ivol_df.columns.tolist()}\n")
print(f"\nK: \n{ivol_df.index.tolist()}\n")

In [ ]:
strikes = ivol_df.iloc[:,0].dropna().index
ivol_df = ivol_df.loc[strikes,:].copy()
T = ivol_df.columns.tolist()
K = ivol_df.index.tolist()
ivol_df

In [ ]:
ivol_array = ivol_df.to_numpy()
ivol_array

In [ ]:
x = np.arange(0, ivol_array.shape[1])
y = np.arange(0, ivol_array.shape[0])
#mask invalid values
array = np.ma.masked_invalid(ivol_array)
xx, yy = np.meshgrid(x, y)
#get only the valid values
x1 = xx[~array.mask]
y1 = yy[~array.mask]
newarr = array[~array.mask]

GD1 = interpolate.griddata((x1, y1), newarr.ravel(),
                          (xx, yy),
                            method='cubic')

GD1

In [ ]:
vol_surf = pd.DataFrame(
    ivol_array,
    index = K,
    columns = T
).copy()

vol_surf

In [ ]:
vol_surf = ivol_df.loc[:,ivol_df.columns>0].dropna(how='any', axis=1).copy()
K = vol_surf.index.tolist()
T = vol_surf.columns.tolist()
vol_surf

In [ ]:
vol_matrix = ql.Matrix(len(K),len(T),0.0)
for i,k in enumerate(K):
    for j,t in enumerate(T):
        vol_matrix[i][j] = float(vol_surf.loc[k,t])

print(vol_matrix)

In [ ]:
bicubic_vol = ql.BicubicSpline(T,K,vol_matrix)

In [ ]:
K = np.linspace(
    min(K),
    max(K),
    50
)
T = np.linspace(
    1,
    10,
    50
)

KK,TT = np.meshgrid(K,T)

V = np.array(
    [[bicubic_vol(float(t),float(k),False) for k in K] for t in T]
    )

plt.rcParams['figure.figsize']=(7,5)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.view_init(elev=20, azim=120)  
surf = ax.plot_surface(KK,TT,V, rstride=1, cstride=1, cmap=cm.coolwarm,
                linewidth=0.1)
fig.colorbar(surf, shrink=0.3, aspect=5)

ax.set_xlabel("Strike", size=9)
ax.set_ylabel("Maturity", size=9)
ax.set_zlabel("Volatility", size=9)

plt.tight_layout()
plt.show()
plt.cla()
plt.clf()